In [221]:
#import packages
import numpy as np
import pandas as pd
import cv2
import time
import math
cv2.startWindowThread()


0

In [222]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/Daniel_7_top_trim.mp4')
fps = cap.get(cv2.CAP_PROP_FPS) #get fps

if cap.isOpened() == False:
    print("Cannot open video file.")

In [223]:
#saving output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)


In [224]:
#defining size for output videos
output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (1920,1080))
#output2 = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (1920,1080))

In [225]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 90)

In [226]:
#creating empty array
# Create an empty DataFrame with columns for x and y velocities
velocities = pd.DataFrame(columns=['x_velocity', 'y_velocity'])
prev_x, prev_y = None, None
prev_time = None


In [227]:
#defining rgb range to detect orange for now

lower_color_bound = np.array([0,100,178])
upper_color_bound = np.array([255,255,255])

In [229]:
#conversion factor, likely incorrect bc camera and setup need to be calibrated
disc_length = 8.26 #disc length in inches
distance = 36 #distance of disc from camera, rough guess

In [231]:
while True:
    # Get current time
    current_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break
    #regions of interest
    height, width, _ = frame.shape
    roi = frame[200:900, 100:1800]
    
    #object detection
    mask = object_detector.apply(frame)

    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #trying rgb threshold
    mask = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV) 
    mask_rgb = cv2.inRange(mask,lower_color_bound,upper_color_bound)
    rgb_contours, _ = cv2.findContours(mask_rgb, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
       
    
    longest_width = 0
    for cnt in rgb_contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(frame, [cnt], -1, (0,255,0), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            if w > longest_width:
                longest_width = w
            print(longest_width)
            rect_coordinates = np.append(rect_coordinates,[x,y,w,h])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0),3)
            
    if longest_width > 0:
        conversion_rate = disc_length / longest_width
    

    # Check if previous position and time are available
    if prev_x is not None and prev_y is not None and prev_time is not None:
        # Calculate the distance traveled in the x and y directions
        dx = x - prev_x
        dy = y - prev_y

        # Calculate the time interval
        dt = current_time - prev_time

        # Calculate the velocity in the x and y directions
        vx = (dx / dt) 
        vy = (dy / dt)
        
        # Add the velocity values to the DataFrame
        velocities = pd.concat([velocities, new_data])

        vx_mph = vx * conversion_rate
        vy_mph = vy * conversion_rate

        # Display the velocity values on the output video
        cv2.putText(frame, 'X Velocity: {:.2f} mph'.format(vx_mph), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, 'Y Velocity: {:.2f} mph'.format(vy_mph), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # create a new dataframe with the new velocities
        new_data = pd.DataFrame({'x_velocity': [vx_mph], 'y_velocity': [vy_mph]})

    # Update the previous position and time variables
    prev_x, prev_y = x, y
    prev_time = current_time
    
    cv2.imshow("frame", frame)
    #cv2.imshow("roi", roi)
    #cv2.imshow("Mask", mask)
    #cv2.imshow("maskrgb",mask_rgb)
    
    #output.write(roi)
    #output2.write(mask)
    if cv2.waitKey(120) & 0xFF == ord('q'):
        break


cap.release()
output.release()
#output2.release()
cv2.destroyAllWindows()

25
52
82
123
129
128
136
145
153
158
160
160
159
162
164
168
172
173
176
182
184
184
182
187
193
144
56


In [232]:
velocities.to_csv("velocities.csv", index=False)
print(velocities)

    x_velocity  y_velocity
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0     0.000000    0.000000
0 -3758.418995 -435.524948
0     0.000000    7.605301
0     0.000000   16.289109
0     0.000000   13.739671
0    17.849654   12.417151
0    26.640536   12.487751
0    25.083988    8.853172
0    27.581402    3.677520
0    25.786153    1.004655
0    25.116402    0.313955
0    27.165506    0.662573
0    27.412326    0.979012
0    26.634117    0.000000
0    26.050776    0.313865
0    26.224592    0.000000
0    26.114770    0.621780
0    24.882207    0.592434
0    25.156543    0.585036
0    24.736989    0.000000
0    23.526090    0.560145
0    24.151809    0.548905
0    24.885554    0.282790
0    25.556833    0.290419
0    23.928325    0.550076
0    22.660947    0.520941
0    32.337509    0.752035
0    84.614363   24.999698
0     0.000000    0.000000
0